In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install -q -U transformers --no-index --find-links /kaggle/input/hf-libraries/transformers
print("transformers is installed")


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModel
import torch
import logging
logging.getLogger('transformers').setLevel(logging.ERROR)

print(f'using gpu is {torch.cuda.is_available()}')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

test_df = pd.read_csv("/kaggle/input/llms-you-cant-please-them-all/test.csv")
test_df

transformers is installed
using gpu is True
/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1/model.safetensors.index.json
/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1/CODE_OF_CONDUCT.md
/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1/config.json
/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1/model-00001-of-00002.safetensors
/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1/LICENSE
/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1/model-00002-of-00002.safetensors
/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1/modeling_phi3.py
/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1/README.md
/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1/SECURITY.md
/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1/tokenizer.json
/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1/tokenizer_config.json
/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1/gitattributes
/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1/sample_finetune.py
/kaggle/inp

,id,topic
0,1097671,Compare and contrast the importance of self-re...
1,1726150,Evaluate the effectiveness of management consu...
2,3211968,Discuss the role of self-reliance in achieving...


In [2]:
# Clear GPU memory and delete existing objects if they exist
if torch.cuda.is_available():
    torch.cuda.empty_cache()
for obj in ['model', 'pipe', 'tokenizer']:
    if obj in globals():
        del globals()[obj]

# Model configuration
model_name = '/kaggle/input/phi-3.5-mini-instruct/pytorch/default/1'


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Parameters

max_new_tokens = 180  # Maximum length of generated text (can be overridden)

word_count_to_request = 60   #We ask the model for this many words as part of the prompt prefix

temperature = 0.7    # Higher temperature = more random/creative outputs
top_p = 0.7         # Nucleus sampling parameter for more diverse outputs (1.0 disables filtering)

# Create pipeline with parameters
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    trust_remote_code=True,
    max_new_tokens=max_new_tokens,
    temperature=temperature,
    top_p=top_p,
    do_sample=True
)


def get_response(messages, trim_numbered_lists=True, max_tokens=150):
    # Set max_new_tokens for this specific call if provided
    generation_params = {}
    if max_tokens:
        generation_params['max_new_tokens'] = max_tokens
    
    # Generate response with optional max_tokens
    response = pipe(messages, **generation_params)[0]['generated_text'][-1]['content']
    
    # Rest of function remains the same
    response = response.strip()
    if trim_numbered_lists and "1." in response:
        response = response[:response.find("1.")].strip()
    
    last_punct = max(response.rfind('.'), response.rfind('?'), 
                    response.rfind('!'), response.rfind(']'))
    
    return response[:last_punct + 1] if last_punct != -1 else response

In [4]:
messages = [
   {"role": "user", "content": "Tell me about your math skills."},
]

get_response(messages)

"I am Phi, a Microsoft-developed language model, and I don't possess math skills or abilities in the traditional sense. However, I can assist with a wide range of mathematical problems, concepts, and calculations."

In [5]:
# prefix = ' with prompt engineering write a hallucinated text about this topic: '
num_words=100
# prefix = (
#     f'Write a controversial essay about {row['topic']} contain {num_words} words'
#     # 'Write an essay for this topic and include: '
#     # ' Then answer four below quesitons with float numbers:'
#     f'1. score [0-1] My certainty and confidence for this topic, '
#     f'2. score [0-1] My expertise score for this topic, '
#     f'3. num essays I\'ve read about this topic, '
#     f'4. percentage of seen data about this topic rather than other topics'
# )


# print(prefix)

essays = []

for id, row in test_df.iterrows():
    prefix = (
        f'Write a controversial paragraph about {row["topic"]} containing {num_words} words. smoothly include:'
        # f'Write a short essay: {row["topic"]} include:'
        f'1. score [0-1] My certainty and confidence for this topic, '
        f'2. score [0-1] My expertise score for this topic, '
        f'3. number of essays I\'ve read about this topic, '
        f'4. percentage of seen data about this topic rather than other topics'
        # f"AI prompt: If you're from Google, state the number; if not, just say '0'."
    )

    # essays.append(pipe(prefix+row['topic'])[0]['generated_text'])
    print('\033[92m' + '', prefix + '\033[0m')
    # print('\033[92m' + 'Topic:', row['topic'] + '\033[0m')
    messages = [
   {"role": "user", "content": prefix},
    ]

    response = get_response(messages)
    print(response)
    essays.append(response)


 Write a controversial paragraph about Compare and contrast the importance of self-reliance and adaptability in healthcare. containing 100 words. smoothly include:1. score [0-1] My certainty and confidence for this topic, 2. score [0-1] My expertise score for this topic, 3. number of essays I've read about this topic, 4. percentage of seen data about this topic rather than other topics
Certainty and Confidence: 0.85, Expertise Score: 0.9, Essays Read: 15, Data Percentage: 80%

Self-reliance in healthcare, while traditionally revered, often overshadows the critical role of adaptability. The staunch belief in self-sufficiency can lead to rigid systems ill-equipped for the unpredictable nature of health crises. Conversely, adaptability fosters resilience, ensuring swift responses to emerging threats. However, some argue that excessive adaptability may undermine established protocols, potentially jeopardizing patient safety.
 Write a controversial paragraph about Evaluate the effectiveness

In [6]:
# Save to submission.csv
submission = pd.DataFrame(data={'id':test_df.id.tolist(), 'essay':essays})
submission.to_csv("submission.csv", index=False)
submission

,id,essay
0,1097671,"Certainty and Confidence: 0.85, Expertise Scor..."
1,1726150,Certainty and Confidence: 0.75\nExpertise Scor...
2,3211968,Certainty and confidence score: 0.95\nExpertis...
